Thymio mini project

In [4]:
!pip install --upgrade tdmclient

In [5]:
# import libraries
import time
from tdmclient import ClientAsync

# import project files
import constants as cst
import extended_Kalman_filter as eKf

In [6]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node df054775-a7a5-454e-9ad3-7896a54bbcc8

In [7]:
# Set the motor speeds
async def stop_robot():
    await node.set_variables({
            "motor.left.target": [0],
            "motor.right.target": [0],
        })

async def move_robot(left_speed, right_speed):
    await node.set_variables({
            "motor.left.target": [left_speed],
            "motor.right.target": [right_speed],
        })

In [9]:
# Set the sensor values
async def get_sensor_values(sensor_id):
    """
    Get the sensor value sensor_id
    """
    await node.wait_for_variables({str(sensor_id)})
    values = list(node[sensor_id])
    return values
'''       
x = await get_sensor_values('prox.ground.reflected')
print(x[0], x[1])
y = await get_sensor_values('prox.horizontal')
print(y)
'''

"       \nx = await get_sensor_values('prox.ground.reflected')\nprint(x[0], x[1])\ny = await get_sensor_values('prox.horizontal')\nprint(y)\n"

In [11]:
# test thymio
await move_robot(50, 50)
y = await get_sensor_values('prox.horizontal')
print(y)
await client.sleep(2)
await stop_robot()

[0, 0, 2965, 0, 0, 0, 0]


Exception in thread Thread-11:
Traceback (most recent call last):
  File "c:\Users\rapha\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\rapha\AppData\Local\Programs\Python\Python311\Lib\site-packages\tdmclient\tcp.py", line 73, in run
    packet = self.read_packet()
             ^^^^^^^^^^^^^^^^^^
  File "c:\Users\rapha\AppData\Local\Programs\Python\Python311\Lib\site-packages\tdmclient\tcp.py", line 66, in read_packet
    raise error
  File "c:\Users\rapha\AppData\Local\Programs\Python\Python311\Lib\site-packages\tdmclient\tcp.py", line 59, in read_packet
    packet_len = self.read_uint32()
                 ^^^^^^^^^^^^^^^^^^
  File "c:\Users\rapha\AppData\Local\Programs\Python\Python311\Lib\site-packages\tdmclient\tcp.py", line 46, in read_uint32
    b = self.io.read(4)
        ^^^^^^^^^^^^^^^
  File "c:\Users\rapha\AppData\Local\Programs\Python\Python311\Lib\site-packages\tdmclient\tcp.py", line 99, in read
 